# Supporting Information

This notebook is available as an html file or as an executable jupyter notebook on github.

## Table of Contents

* [Load external code](#Load-external-code)
* [Model building framework](#Model-building-framework)
    * [Example model](#Example-model)
    * [Caveat](#Caveat)
    * [Set up fit functionality](#Set-up-fit-functionality)
* [Ensemble FRET dimerization model](#Ensemble-FRET-dimerization-model)
    * [Import data from Chapter 4, Figure 4: Dimerization of FRET-labeled bZIP](#Import-data-from-Chapter-4,-Figure-4:-Dimerization-of-FRET-labeled-bZIP)
    * [Define the parameters of the model](#Define-the-parameters-of-the-model-(1))
    * [Testing fit robustness](#Testing-fit-robustness)
* [Stepwise affinity constants for DNA binding](#Stepwise-affinity-constants-for-DNA-binding)
    * [DNA-driven dimerization model](#DNA-driven-dimerization-model)
        * [Import data from Chapter 5, Figure 5.3J: DNA binding by CREB bZIP](#Import-data-from-Chapter-5,-Figure-5.3J:-DNA-binding-by-CREB-bZIP)
        * [Define the parameters of the model](#Define-the-parameters-of-the-model-(2))
        * [bZIP fit](#bZIP)
        * [FL fit](#FL)
        * [Plots](#smFRET-fit-plots)
    * [Fluorescence anisotropy model](#Fluorescence-anisotropy-model)
        * [Import data from Chapter 4, Figure 4.3: Fluorescence anisotropy experiments of bZIP binding to nsDNA](#Import-data-from-Chapter-4,-Figure-4.3:-Fluorescence-anisotropy-experiments-of-bZIP-binding-to-nsDNA)
        * [Define the parameters and fit the model](#Define-the-parameters-and-fit-the-model)
* [Simulations of stepwise binding constants](#Simulations-of-stepwise-binding-constants)
    
## Load external code

In [1]:
from inspect import signature
import warnings

import numpy as np
import pandas as pd
from scipy.optimize import fsolve
import lmfit

from wrangling.bokeh_scatter import scatter_palette as palette
import bokeh.plotting
import bokeh.io

bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
# this code makes plots look nice


def format_bokeh_plot(plot, grid=False):
    """Consistent in-place formatting of bokeh plots."""
    
    if grid == False:
        plot.grid.grid_line_color = None

    plot.axis.axis_line_width = 2
    plot.axis.axis_line_color = "black"
    plot.axis.axis_line_alpha = 1
    plot.axis.major_tick_line_width = 2
    plot.axis.major_tick_in = 0
    plot.axis.minor_tick_out = 0

    plot.axis.axis_label_text_font_size = "16pt"
    plot.axis.major_label_text_font_size = "14pt"
    plot.axis.axis_label_text_font_style = "bold"
    plot.axis.major_label_text_font_style = "bold"
    plot.outline_line_color = None

    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        plot.legend.label_text_font_size = "14pt"
        plot.title.text_font_size = "18pt"

    return

----
## Model building framework 

In this code, we will be working with a variety of models all built on the same framework. The function `make_model_function` provides this framework. A list of `equations` describing the system to be modeled is provided to `make_model_function`. The `model` that is returned has the `equations` and the x variable, called `axis`, built into it, and calculates values based on provided paramaters called `knowns`.

The `equations` provided to `make_model_function` are simple: they are the "conservation of mass" equations of the system being described, rearranged to equal zero, plus the value of interest to calculate.

For example, given the interaction:

$a + b \leftrightarrow ab$

$K_d = \frac{[a][b]}{[ab]}$

We have the conservation of mass equations:

$0 = [a] + \frac{[a][b]}{K_d} - [a]_{total}$

$0 = [b] + \frac{[a][b]}{K_d} - [b]_{total}$

We may wish to calculate:

$binding fraction = \frac{[b]}{K_d + [b]}$

Rather than algebraically calculate the unknown concentrations of the free molecules $a$ and $b$, we simply provide the above equations to `make_model_function` as described in its docstring.
```
equations = lambda a_tot, a, b_tot, b, Kd : [
            a + a*b / Kd - a_tot,
            b + a*b / Kd - b_tot,
            b / (Kd + b)
        ]
```

This example is simple enough that algebraically calculating the unknown concentrations is possible, but this approach is viable for models where such algebra is not tractable.

The resulting model relies on fsolve, a function of the scipy.optimize package, to calculate unknown values. fsolve uses numerical root solving to find the unknowns in the conservation of mass equations: essentially, it guesses values for the unknowns and checks how well they satisfy the equations, then iteratively adjusts its guesses until it converges on a solution.

In [3]:
# this function is important for models with more than one independent variable


def structure_axis_points(points, number_of_axes):
    """Guarantee that axis_points has the right format to pass to model.
    Return a sorted list of tuples of identical lengths equal to the number of
    axes. The list is merely a collection of those coordinates and can be any
    length.
    """

    # possible points inputs:
    # one value: 1, number_of_axes=1
    # collection along one axis: [1, 2, 3], number_of_axes=1
    # one multi-axis set of values: (1, 10), number_of_axes=2
    # collection along multiple axes:
    # [(1, 10), (2, 20), (3, 30)], number_of_axes=2

    array = np.array(points)

    try:
        if array.ndim == 0:
            assert number_of_axes == 1
            points = [(points,)]
        elif array.ndim == 1:
            if number_of_axes in array.shape:
                print(
                    "Interpreting points as a specification of a single point."
                )
                points = [points]
            else:
                assert number_of_axes == 1
                points = [(point,) for point in points]
        elif array.ndim > 1:
            assert number_of_axes == array.shape[-1]
        else:
            raise RuntimeError(
                "You provided an array with negative or non-integer dimensions?"
            )

    except AssertionError:
        raise RuntimeError(
            "The number of axes in the model does not match the dimensions of "
            + "the points provided."
        )

    points.sort()
    return points


# this function is the actual model framework


def make_model_function(equations, axis):
    """Make a function to model a system of equations.

    Parameters
    ----------
    equations : func
        this function should contain a list of equations describing the system.
        All equations but the last should be conservation of mass equations,
        which are true when set to zero. The last equation in the list should
        calculate the value of interest that the model will return. The
        function will return a list of values calculated from the list of
        equations. When the returned values {equations(*args)[:-1]} equal zero,
        the last value {equations(*args)[-1]} represents the value of interest
        for the given inputs.
    axis : str or list containing argument(s) of equations
        the variable(s) that will be varied in the model, i.e. the x-axis

    Returns
    -------
    a function that accepts the arguments:
        axis_points (will be passed as the 'axis' variable(s))
        **kwargs (as defined by the arguments taken by equations, as provided to
            make_model_function)
        ymin (default None)
        ymax (default None)

    Example
    -------
    Consider the system:
    a + b -> ab with dissociation constant Kd
    variables will represent the concentration of the corresponding molecule
    (note that a and b represent the free unbound state of those molecules)
        Kd = a*b / ab, so ab = a*b / Kd
        a_total = a + ab = a + a*b / Kd
        b_total = b + ab = b + a*b / Kd
    the a_total and b_total equations can be rearranged to equal zero. These
    are the conservation of mass equations:
        0 = a + a*b / Kd - a_total
        0 = b + a*b / Kd - b_total

    I want to calculate molecules of ab per molecules of a total:
    ab / (a + ab) = b / (Kd + b)
    This expression will be the last equation in the list.

    I construct the argument to be passed as follows:
        equations = lambda a_total, a, b_total, b, Kd : [
            a + a*b / Kd - a_total,
            b + a*b / Kd - b_total,
            b / (Kd + b)
        ]

    I want to model titrations as b_total is varied.
    The full call looks like:

    model = make_model_function(
        equations = lambda a_total, a, b_total, b, Kd : [
            a + a*b / Kd - a_total,
            b + a*b / Kd - b_total,
            b / (Kd + b)
        ],
        axis="b_total"
    )

    The return will be a function (here called "model") that will accept a
    positional argument axis_points (corresponding to the "axis" variable, here
    b_total), and any of (a_total, a, b, Kd) as keyword arguments. Variables
    that are constant as x varies should be provided as knowns (in this case,
    a_total and Kd), while variables that change as x varies are calculated as
    unknowns (in this case, the free concentrations a and b). Thus, I can call
    the model:

    b_tot_points = [x for x in range(100)]
    curve = model(b_tot_points, a_total=50, Kd=0.5)

    which will return an array of the same length as b_tot_points, containing
    the calculated value from the last equation provided to make_model_function
    (here, the molecules of ab per molecules of a total) at each b_total
    point and with the known values provided (a_total=50, K=0.2). A titration
    curve can be constructed with x_values = b_tot_points and y_values = curve.

    All model functions returned from make_model_function() accept the
    keyword arguments "ymin" and "ymax", allowing arbitrary scaling of the
    returned values (and thus the y axis of the titration curve). To use them,
    both must be specified.
    """

    if type(axis) is str:
        axis = [axis]

    def model(axis_points, **knowns):
        try:
            ymin = knowns.pop("ymin")
            ymax = knowns.pop("ymax")
        except KeyError:
            ymin = None
            ymax = None

        axis_points = structure_axis_points(axis_points, len(axis))

        for var in axis:
            if var in knowns.keys():
                warnings.warn(
                    f"\n{var} must be provided as a coordinate in axis_points. "
                    + "Ignoring keyword argument."
                )

        unknowns = []
        for param in signature(equations).parameters:
            if param not in knowns.keys() and param not in axis:
                unknowns.append(param)

        def to_solve(array, axis_value):
            """This function will be used iteratively by fsolve.
            array contains the current guesses for unknown variables.
            """
            # assign each array value to an unknown
            kwargs = {var: array[i] for i, var in enumerate(unknowns)}
            # add knowns to kwargs
            kwargs.update(knowns)
            for i, var in enumerate(axis):
                kwargs[var] = axis_value[i]

            # calculate the outputs of equations based on guessed values
            # the guesses are accurate when the outputs are all zero
            # the last equation is for the final return - drop for root solving
            return np.array(equations(**kwargs)[:-1])

        results = []
        # initialize starting point
        guess_solutions = [1] * len(unknowns)

        # axis_points is a list of tuples
        # axis_value will be a tuple
        # although it may have len 1

        for axis_value in axis_points:
            """For each axis_value, fsolve will provide guess_solutions as an
            array to to_solve and iteratively update the guesses until to_solve
            returns values as close to zero as possible. The final guesses are
            recorded as solutions.
            """
            # last comma is MANDATORY or the tuple gets unpacked
            solutions = fsolve(to_solve, guess_solutions, args=(axis_value,))

            # record the solutions at the last x_value
            # to use as starting points for the next x_value
            # this significantly improves stability and speed
            guess_solutions = solutions

            # combine the unknown solutions with the known variables
            solutions_dict = {
                variable: value for variable, value in zip(unknowns, solutions)
            }
            solutions_dict.update(**knowns)
            for i, var in enumerate(axis):
                solutions_dict[var] = axis_value[i]

            # calculate the value of interest from the last provided equation
            # and save it to results, corresponding to the current axis_value
            results.append(equations(**solutions_dict)[-1])

            # uncommenting this 'print' line will print all calculated variables
            # for every x_value
            # print(solutions_dict)

        curve = np.array(results, dtype=float)
        # normalize curve to ymin and ymax, if provided
        # and return full list of results
        if ymin is not None:
            curve = curve - curve.min()
            curve = curve / curve.max()
            return curve * (ymax - ymin) + ymin
        else:
            return curve

    # this code runs when model is being defined
    # and adds a minimal descriptive docstring
    params = signature(equations)
    arguments = (
        str(signature(equations))
        .replace("(", "")
        .replace(")", "")
        .replace(" ", "")
        .split(",")
    )
    arguments = [x for x in arguments if x not in axis]
    arguments.append("ymin")
    arguments.append("ymax")

    model.__doc__ = (
        f"Possible arguments to be passed by keyword: "
        + f"{arguments}\n{axis} is/are the independent variable(s)."
    )

    return model

### Example model

$a + b \leftrightarrow ab$

$K_d = 10$

Note that we do not need separate models for Simple and Quadratic forms of these equations. No assumptions are made about free $b$ being equivalent to $b_{total}$. The same model can handle negligible and non-negligible concentrations of $a_{total}$. Here we will calculate values for a_total = 1 and a_total =20.

In [4]:
model = make_model_function(
    equations=lambda a_total, a, b_total, b, Kd: [
        a + (a * b / Kd) - a_total,
        b + (a * b / Kd) - b_total,
        b / (Kd + b),
    ],
    axis="b_total",
)

b_tot_points = np.logspace(-1, 3, 100)
curve_atot_1 = model(b_tot_points, a_total=0.1, Kd=10)
curve_atot_20 = model(b_tot_points, a_total=20, Kd=10)

pd.DataFrame(
    {
        "b_total": b_tot_points,
        "fraction a bound, a_tot=1": curve_atot_1,
        "fraction a bound, a_tot=20": curve_atot_20,
    }
)

,b_total,"fraction a bound, a_tot=1","fraction a bound, a_tot=20"
0,0.100000,0.009805,0.003330
1,0.109750,0.010751,0.003654
2,0.120450,0.011787,0.004010
3,0.132194,0.012921,0.004400
4,0.145083,0.014163,0.004828
...,...,...,...
95,689.261210,0.985697,0.985284
96,756.463328,0.986951,0.986608
97,830.217568,0.988097,0.987812
98,911.162756,0.989143,0.988906


In [5]:
linear = bokeh.plotting.Figure(
    x_axis_label="b total",
    y_axis_label="fraction a bound",
    height=400,
    width=400,
)
log = bokeh.plotting.Figure(
    x_axis_label="b total", height=400, width=400, x_axis_type="log"
)

for name, plot in [("linear", linear), ("log", log)]:
    plot.circle(
        b_tot_points, curve_atot_1, color=palette[0], legend_label="a_tot = 1"
    )
    plot.circle(
        b_tot_points, curve_atot_20, color=palette[1], legend_label="a_tot = 20"
    )

    plot.title.text = f"{name} scale"
    format_bokeh_plot(plot)
    plot.legend.location = "bottom_right"

bokeh.io.show(bokeh.layouts.row([linear, log]))

### Caveat

Numerical root solving relies on the initial guesses provided being reasonable. In edge cases involving extremely high or low values, we can obtain nonsensical results. Here we try to ask the model for a binding curve where `a_total` is 100 times higher than `Kd`. Even though we know the values for "fraction a bound" should vary between 0 and 1, the model's initial guesses in solving this edge case are so far off that its best solutions are disconnected from reality.

In [6]:
curve = model(b_tot_points, a_total=50, Kd=0.5)

linear = bokeh.plotting.Figure(
    x_axis_label="b total",
    y_axis_label="fraction a bound",
    height=400,
    width=400,
)
log = bokeh.plotting.Figure(
    x_axis_label="b total", height=400, width=400, x_axis_type="log"
)

for plot in [linear, log]:
    plot.circle(b_tot_points, curve, color=palette[0])
    format_bokeh_plot(plot)

bokeh.io.show(bokeh.layouts.row([linear, log]))

### Set up fit functionality

To use the models we will create in fitting, we will need to be able to calculate residuals, i.e. the differences between a model and a dataset. These functions were written to be compatible with the `lmfit` package, which performs Non-Linear Least-Squares minimization and curve-fitting.

In [7]:
def calculate_residuals(params, model, x_values, y_values, error=None, i=None):
    """Calculates residuals between a model with given parameters and a dataset.

    Parameters
    ----------
    params : an lmfit.Parameters object
    model : func
        the model function to fit
    x_values : array-like
        the x values of the data
    y_values : array-like
        the y values of the data
    error : array-like, optional
        the standard deviations or other weights to apply to the data
    i : int
        an iterator to be used by multidimensional_residuals() when globally
        fitting multiple datasets
    """

    if error is None:
        error = np.ones(np.array(x_values).shape)

    if i is None:
        curve = model(x_values, **params)
    else:
        # select only parameters ending in _{i} and chop that ending off them
        sub_params = {
            param[: param.rfind("_")]: value
            for param, value in params.valuesdict().items()
            if str(i) in param
        }
        curve = model(x_values, **sub_params)

    return (y_values - curve) / error


def multidimensional_residuals(params, model, df):
    """Calculates residuals for multiple datasets.

    Parameters
    ----------
    params : lmfit.Parameters object
        should contain parameters with the form {param_name}_{i},
        where i corresponds to the dataset for that parameter, ordered
        as in df.columns.levels[0]
    model : func
        calculates residuals for a single dataset
    df : pandas DataFrame
        index should be x values
        columns should be a multiindex, where
            level 1 specifies the various datasets
            level 2 contains "mean" and "std" columns
    """

    arrays = []

    try:
        for i, dataset in enumerate(df.columns.levels[0]):

            dataset_residuals = calculate_residuals(
                params,
                model,
                df.index.values,
                df[dataset, "mean"].values,
                df[dataset, "std"].values,
                i,
            )
            dataset_residuals = dataset_residuals[~np.isnan(dataset_residuals)]
            arrays.append(dataset_residuals)

    except AttributeError:
        for i, colname in enumerate(df.columns):
            dataset_residuals = calculate_residuals(
                params,
                model,
                df.index.values,
                df[colname].values,
                error=None,
                i=i,
            )
            dataset_residuals = dataset_residuals[~np.isnan(dataset_residuals)]
            arrays.append(dataset_residuals)

    return np.concatenate(arrays)

----

## Ensemble FRET dimerization model

This model describes the ratio of green-labeled monomer fluorescence observed before and after a 2x dilution with varying concentrations of red-labeled monomer (Chapter 3, Figure 3.4). All three possible dimerization interactions are assumed to have the same affinity.

$G + G \leftrightarrow D_G$

$R + R \leftrightarrow D_R$

$G + R \leftrightarrow D_F$

$K_d = \frac{[G]^2}{[D_G]} = \frac{[R]^2}{[D_R]} = \frac{[G][R]}{[D_F]}$

We describe the final (after dilution) concentrations of G and R labeled protein as $G_{total}$ and $R_{total}$. The total concentration of G labeled protein before dilution is $2G_{total}$. (The total concentration of red labeled protein before dilution is zero.)

The conservation of mass equations are:

$0 = [G] + \frac{[G][R]}{K_d} + 2\frac{[G]^2}{K_d} - [G]_{total}$

$0 = [R] + \frac{[G][R]}{K_d} + 2\frac{[R]^2}{K_d} - [R]_{total}$

The value we want to calculate is:

$ratio = \frac{2[G]^2 + [G](K_d + q[R])}{2K_d[G]_{total}}$

where $q$ is the proportion of fluorescence quenched upon FRET dimerization.

In [8]:
fret_dimer_model = make_model_function(
    equations=lambda G_total, R_total, Kd, q, G, R: [
        G + G * R / Kd + 2 * G ** 2 / Kd - G_total,
        R + G * R / Kd + 2 * R ** 2 / Kd - R_total,
        (G * Kd + q * G * R + 2 * G ** 2) / (2 * Kd * G_total),
    ],
    axis="R_total",
)

fret_dimer_model.__doc__ += (
    "\nCalculates the fluorescence intensity ratio of a FRET dimer titration."
    "\n\nParameters"
    "\n----------"
    "\nR_total : total red-labeled monomer concentration (independent variable)"
    "\nG_total : total green-labeled monomer concentration after dilution"
    "\nKd : dissociation constant for dimerization"
    "\nqD : quenching upon G homodimerization"
    "\nqF : quenching upon FRET dimerization"
    "\nG : free green-labeled monomer concentration"
    "\nR : free red-labeled monomer concentration"
    "\nGb : free green-labeled monomer concentration before dilution"
)

### Import data from Chapter 4, Figure 4: Dimerization of FRET-labeled bZIP

In [9]:
dimer_FRET = pd.DataFrame(
    {
        "mean": [
            0.4287205,
            0.43383718,
            0.426269195,
            0.416366121,
            0.417448872,
            0.408032872,
            0.386507997,
            0.353594002,
            0.262869044,
            0.18712771,
            0.145013256,
            0.147505,
            0.125832736,
            0.097707444,
            0.096466104,
        ],
        "std": [
            0.0057315,
            0.03299232,
            0.028265668,
            0.029385818,
            0.030924393,
            0.027176427,
            0.033236957,
            0.026721022,
            0.023179696,
            0.015987862,
            0.010452981,
            0,
            0.022910388,
            0.004803442,
            0,
        ],
        "N": [
            2,
            4,
            4,
            4,
            4,
            4,
            4,
            4,
            4,
            4,
            4,
            1,
            3,
            2,
            1,
        ],
    },
    index=[
        0,
        0.97,
        1.95,
        3.9,
        7.8,
        15.6,
        31.3,
        62.5,
        125,
        250,
        500,
        680,
        1000,
        2000,
        4000,
    ],
)

dimer_FRET.index.name = "[A647 bZIP] (nM)"


def plot_dimer_FRET_data(height=400, width=600):

    alpha = 0.75
    size = 12

    p = bokeh.plotting.Figure(
        x_axis_label="[A647 bZIP] (nM)",
        y_axis_label="Intensity ratio",
        x_axis_type="log",
        x_range=(0.1, 10000),
        y_range=(0, 0.6),
        height=height,
        width=width,
    )

    blue_data = dimer_FRET.loc[dimer_FRET["N"] == 4]
    grey_data = dimer_FRET.loc[dimer_FRET["N"] != 4]

    # plot blue data
    p.circle(
        blue_data.index,
        blue_data["mean"],
        color=palette[0],
        size=size,
        alpha=alpha,
    )
    for x, y, sigma in zip(blue_data.index, blue_data["mean"], blue_data["std"]):
        p.line([x, x], [y - sigma, y + sigma], color=palette[0])

    # plot grey data
    p.circle(
        grey_data.index, grey_data["mean"], color="grey", size=size, alpha=alpha
    )
    for x, y, sigma in zip(grey_data.index, grey_data["mean"], grey_data["std"]):
        p.line([x, x], [y - sigma, y + sigma], color="grey")

    # plot empty first point
    p.circle(
        grey_data.index.values[0],
        grey_data["mean"].values[0],
        color="white",
        line_color="grey",
        size=size,
    )
    p.line(
        [grey_data.index.values[0]] * 2,
        [
            grey_data["mean"].values[0] - grey_data["std"].values[0],
            grey_data["mean"].values[0] + grey_data["std"].values[0],
        ],
        color="grey",
    )

    return p


p = plot_dimer_FRET_data()
format_bokeh_plot(p)
bokeh.io.show(p)

### Define the parameters of the model (1)

We use `lmfit` to define our parameters and fit models. For this data, `G_total` will be fixed at a value of 5, while `Kd` and `q` will be allowed to vary.

In [10]:
params = lmfit.Parameters()

params.add("G_total", value=5, vary=False)
params.add("Kd", value=100, min=0)
params.add("q", value=0.2, min=0, max=1)

results = lmfit.minimize(
    calculate_residuals,
    params,
    args=(fret_dimer_model, dimer_FRET.index, dimer_FRET["mean"]),
)
results

In [11]:
# plot fit curve
smooth_x = np.logspace(-1, 4, 100)
curve = fret_dimer_model(smooth_x, **results.params)
p = plot_dimer_FRET_data()
p.line(smooth_x, curve, color=palette[0])
p.title.text = (
    f"Kd = {round(results.params['Kd'].value, 1)}, "
    + f"q = {round(results.params['q'].value*100, 1)}%"
)
format_bokeh_plot(p)

bokeh.io.show(p)

The discrepancy between the model and the fit at low concentrations of A647 bZIP is due to adsorption and is discussed in more detail in Chapter 4.

### Testing fit robustness

We can show that this fit is robust by repeating it with a variety of initial values.

In [12]:
for i, (Kd, q) in enumerate(
    [
        (100, 0.1),
        (50, 0.1),
        (10, 0.1),
        (100, 0.5),
        (50, 0.5),
        (10, 0.5),
        (100, 0.9),
        (50, 0.9),
        (10, 0.9),
    ]
):

    params["Kd"].value = Kd
    params["q"].value = q

    results = lmfit.minimize(
        calculate_residuals,
        params,
        args=(fret_dimer_model, dimer_FRET.index, dimer_FRET["mean"]),
    )

    print(f"Test {i+1}")
    print(
        f"Initial Kd = {params['Kd'].value}, "
        + f"fit Kd = {round(results.params['Kd'].value, 2)}"
    )
    print(
        f"Initial q = {params['q'].value * 100}%, "
        + f"fit q = {round(results.params['q'].value*100, 2)}%"
    )
    print()

Test 1
Initial Kd = 100, fit Kd = 27.98
Initial q = 10.0%, fit q = 6.06%

Test 2
Initial Kd = 50, fit Kd = 27.98
Initial q = 10.0%, fit q = 6.06%

Test 3
Initial Kd = 10, fit Kd = 27.98
Initial q = 10.0%, fit q = 6.06%

Test 4
Initial Kd = 100, fit Kd = 27.98
Initial q = 50.0%, fit q = 6.06%

Test 5
Initial Kd = 50, fit Kd = 27.98
Initial q = 50.0%, fit q = 6.06%

Test 6
Initial Kd = 10, fit Kd = 27.98
Initial q = 50.0%, fit q = 6.06%

Test 7
Initial Kd = 100, fit Kd = 27.98
Initial q = 90.0%, fit q = 6.06%

Test 8
Initial Kd = 50, fit Kd = 27.98
Initial q = 90.0%, fit q = 6.06%

Test 9
Initial Kd = 10, fit Kd = 27.98
Initial q = 90.0%, fit q = 6.06%



We can also test whether the fit values can compensate for one another by fixing them at different values, one at a time. For this fit to be considered as having truly converged, we should see worse fits when q or Kd are fixed at values far from their fit values.

In [13]:
alpha = 0.75
size = 12
smooth_x = np.logspace(-1, 4, 100)

plots = []

for variable, value in [
    ("Kd", 1),
    ("Kd", 100),
    ("Kd", 1000),
    ("q", 0),
    ("q", 0.2),
    ("q", 0.5),
]:

    params = lmfit.Parameters()

    params.add("G_total", value=5, vary=False)
    params.add("Kd", value=100, min=0)
    params.add("q", value=0.2, min=0, max=1)

    params[variable].value = value
    params[variable].vary = False

    p = plot_dimer_FRET_data(300, 450)

    results = lmfit.minimize(
        calculate_residuals,
        params,
        args=(fret_dimer_model, dimer_FRET.index, dimer_FRET["mean"]),
    )

    curve = fret_dimer_model(smooth_x, **results.params)
    p.line(smooth_x, curve, color=palette[0])
    p.title.text = (
        f"{variable} fixed: Kd = {round(results.params['Kd'].value, 1)}, "
        + f"q = {round(results.params['q'].value*100, 1)}%"
    )

    format_bokeh_plot(p)
    plots.append(p)

bokeh.io.show(bokeh.layouts.grid(plots, ncols=3))

C:\Users\Emily\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


From these results, we can conclude that the fit is robust.

----

## Stepwise affinity constants for DNA binding

This model describes the following pair of binding steps:

$M + M \leftrightarrow M_2$

$M_2 + DNA \leftrightarrow M_2DNA$

We define:

$K_{d,dim} = K_{d3} = \frac{[M]^2}{[M_2]}$

$K_{d,DNA} = K_{d4} = \frac{[M_2][DNA]}{[M_2DNA]}$

($K_{d1}$ and $K_{d2}$ refer to $K_{d,first M}$ and $K_{d,second M}$, respectively.)

In chapters 4 and 5, we directly measure $K_{d,dim}$ in the absence of DNA. This allows us to fix this value in order to fit $K_{d,DNA}$.

We assume that $M_1DNA$ has a concentration of 0. The conservation of mass equations are:

$0 = [M] + 2\frac{[M]^2}{K_{d3}} + 2\frac{[M]^2[DNA]}{K_{d3}K_{d4}} - [M]_{total}$

$0 = [DNA] + \frac{[M]^2[DNA]}{K_{d3}K_{d4}} - [DNA]_{total}$

### DNA-driven dimerization model

First, we discuss DNA-driven dimerization measured by smFRET. The value we want to calculate is:

<font size="4">$f_D = \frac{2\frac{[M]}{K_{d3}}+2\frac{[M][DNA]}{K_{d3}K_{d4}}}{1+2\frac{[M]}{K_{d3}}+2\frac{[M][DNA]}{K_{d3}K_{d4}}}$</font>

In [14]:
FRET_DNA_model = make_model_function(
    equations=lambda M_total, DNA_total, Kd3, Kd4, M, DNA: [
        M + 2 * (M ** 2) / Kd3 + 2 * (M ** 2) * DNA / (Kd3 * Kd4) - M_total,
        DNA + (M ** 2 * DNA / (Kd3 * Kd4)) - DNA_total,
        ((2 * M / Kd3) + (2 * M * DNA / (Kd3 * Kd4)))
        / (1 + (2 * M / Kd3) + (2 * M * DNA / (Kd3 * Kd4))),
    ],
    axis="DNA_total",
)

FRET_DNA_model.__doc__ += (
    "\nCalculate dimer fraction with varying DNA concentration."
    "\n\nParameters"
    "\n----------"
    "\nDNA_total : total concentration of DNA (independent variable)"
    "\nM_total : total concentration of monomer"
    "\nKd3 : association constant for M + M -> M(2)"
    "\nKd4 : association constant for M(2) + DNA -> M(2)DNA"
    "\nM : concentration of free monomer. Will be calculated"
    "\nDNA : concentration of free DNA. Will be calculated"
)

### Import data from Chapter 5, Figure 5.3J: DNA binding by CREB bZIP

In [15]:
# note that error is here labeled as "std" in order to fit the expected format
# but error in this dataset is actually the 95% confidence interval of a PDA Fit

FL_smFRET = pd.DataFrame(
    {
        ("no CRE", "mean"): [
            0.464684,
            None,
            None,
            0.352721,
            0.452352,
            None,
            0.569114,
            0.66748,
            0.713851,
            0.731321,
            0.829019,
            0.852205,
            0.931778,
        ],
        ("no CRE", "std"): [
            0.064778,
            None,
            None,
            0.121034,
            0.086345,
            None,
            0.062689,
            0.069563,
            0.067018,
            0.070312,
            0.081862,
            0.080396,
            0.061843,
        ],
        ("half CRE", "mean"): [
            None,
            0.605007,
            0.832155,
            0.820587,
            0.823512,
            0.917264,
            None,
            None,
            None,
            0.953114,
            None,
            None,
            None,
        ],
        ("half CRE", "std"): [
            None,
            0.079223,
            0.109062,
            0.077965,
            0.084548,
            0.094657,
            None,
            None,
            None,
            0.068449,
            None,
            None,
            None,
        ],
        ("full CRE", "mean"): [
            None,
            0.774539,
            0.926227,
            0.881795,
            0.861237,
            0.942972,
            None,
            None,
            None,
            0.95689,
            None,
            None,
            None,
        ],
        ("full CRE", "std"): [
            None,
            0.110866,
            0.107409,
            0.086099,
            0.110244,
            0.087563,
            None,
            None,
            None,
            0.077514,
            None,
            None,
            None,
        ],
    },
    index=[0, 8, 32, 64, 128, 256, 512, 1000, 1280, 2048, 4096, 8192, 16000],
)

bZIP_smFRET = pd.DataFrame(
    {
        ("no CRE", "mean"): [
            0.330129,
            0.271068,
            0.366003,
            0.416947,
            0.542582,
            0.541885,
            0.613872,
            0.691604,
            0.77304,
            0.807496,
            0.851211,
            0.824234,
        ],
        ("no CRE", "std"): [
            0.044155,
            0.082935,
            0.062607,
            0.096417,
            0.085105,
            0.082173,
            0.071588,
            0.087357,
            0.104337,
            0.095029,
            0.089637,
            0.086403,
        ],
        ("half CRE", "mean"): [
            0.354725,
            0.777787,
            0.836356,
            0.926494,
            0.871291,
            0.906889,
            None,
            0.947683,
            None,
            None,
            None,
            None,
        ],
        ("half CRE", "std"): [
            0.055432,
            0.075753,
            0.070713,
            0.105956,
            0.061061,
            0.074913,
            None,
            0.109389,
            None,
            None,
            None,
            None,
        ],
        ("full CRE", "mean"): [
            0.382431,
            0.921827,
            0.915948,
            0.891933,
            None,
            None,
            None,
            0.893807,
            None,
            None,
            None,
            None,
        ],
        ("full CRE", "std"): [
            0.0646,
            0.057309,
            0.077147,
            0.06856,
            None,
            None,
            None,
            0.153151,
            None,
            None,
            None,
            None,
        ],
    },
    index=[
        0,
        8,
        16,
        32,
        64,
        128,
        256,
        512,
        1024,
        2048,
        4096,
        8192,
    ],
)

bZIP_smFRET.index.name = "[DNA] (nM)"
FL_smFRET.index.name = "[DNA] (nM)"

FRET_palette = ["#7e1f7c", "#e287bb", "#606060"]

smFRET = {"bZIP": bZIP_smFRET, "FL": FL_smFRET}


def plot_smFRET_data(construct="bZIP", height=400, width=600, size=8):

    p = bokeh.plotting.Figure(
        x_axis_type="log",
        x_axis_label="[DNA] (nM)",
        y_axis_label="Dimer fraction",
        height=height,
        width=width,
        y_range=(0, 1.1),
        x_range=(1, 100000),
    )

    for i, DNA in enumerate(smFRET[construct].columns.levels[0]):
        p.circle(
            smFRET[construct][DNA].dropna().index,
            smFRET[construct][DNA, "mean"].dropna(),
            color=FRET_palette[i],
            legend_label=DNA,
            size=size,
        )

        # plot error bars
        for _, row in smFRET[construct][DNA].dropna().iterrows():
            p.line(
                [row.name] * 2,
                [row["mean"] - row["std"], row["mean"] + row["std"]],
                color=FRET_palette[i],
                legend_label=DNA,
            )

    return p


p = plot_smFRET_data("bZIP")
p.title.text = "bZIP"
p.legend.location = "bottom_right"
format_bokeh_plot(p)

q = plot_smFRET_data("FL")
q.title.text = "FL"
q.legend.location = "bottom_right"
format_bokeh_plot(q)

bokeh.io.show(bokeh.layouts.column([p, q]))

### Define the parameters of the model (2)

Both bZIP and FL experiments can use the same parameters. Each experiment includes three datasets, so we will set up three of each parameter. Paramaters for full CRE DNA will have the suffix \"_0\"; parameters for half CRE DNA will have the suffix \"_1\"; and parameters for nonspecific DNA will have the suffix parameters for half CRE DNA will have the suffix \"_2\". We will restrict the ymin and ymax to be equal between datasets, but allow different Kd4 values. We also set M_total to be a constant.

We set $K_{d3}$ to be equal to 33 nM, the value measured for bZIP by smFRET in Chapter 5. The fit can also be run with $K_{d3} =$ 15 nM, the value measured by mass photometry in Chapter 5.

In [16]:
FRET_params = lmfit.Parameters()

for i, _ in enumerate(smFRET["bZIP"].columns.levels[0]):
    # this line defines the dimerization Kd
    FRET_params.add(f"Kd3_{i}", value=33, min=12, max=55, vary=False)
    FRET_params.add(f"Kd4_{i}", min=0)
    FRET_params.add(f"M_total_{i}", value=10, vary=False)
    FRET_params.add(f"ymin_{i}", value=0.3)
    FRET_params.add(f"ymax_{i}", value=0.95, min=0, max=1)

    if i > 0:
        FRET_params[f"Kd3_{i}"].expr = "Kd3_0"
        FRET_params[f"ymin_{i}"].expr = "ymin_0"
        FRET_params[f"ymax_{i}"].expr = "ymax_0"

# use separate predictions for each DNA sequence
FRET_params["Kd4_0"].value = 0.1
FRET_params["Kd4_1"].value = 1
FRET_params["Kd4_2"].value = 50

### Fit the model

#### bZIP

In [17]:
results = {}
results["bZIP"] = lmfit.minimize(
    multidimensional_residuals,
    FRET_params,
    args=(FRET_DNA_model, smFRET["bZIP"]),
)

results["bZIP"]

C:\Users\Emily\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\Emily\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last five Jacobian evaluations.
  warnings.warn(msg, RuntimeWarning)


## FL

In [18]:
results["FL"] = lmfit.minimize(
    multidimensional_residuals, 
    FRET_params, 
    args=(FRET_DNA_model, smFRET["FL"])
)

results["FL"]

#### smFRET fit plots

In [19]:
plots = []
smooth_x = np.logspace(-2, 6, 1000)

for construct, data in smFRET.items():

    p = plot_smFRET_data(construct, size=10)

    curves = {}
    result = results[construct]

    for i, DNA in enumerate(data.columns.levels[0]):
        kwargs = {
            "Kd3": result.params[f"Kd3_{i}"],
            "Kd4": result.params[f"Kd4_{i}"],
            "M_total": result.params[f"M_total_{i}"],
            "ymin": result.params[f"ymin_{i}"],
            "ymax": result.params[f"ymax_{i}"],
        }

        curve = FRET_DNA_model(smooth_x, **kwargs)
        p.line(
            smooth_x,
            curve,
            color=FRET_palette[i],
            legend_label=str(DNA),
            line_width=3,
        )
        curves[DNA] = curve

    p.legend.location = "bottom_right"
    p.title.text = " ".join([
        f"{construct}: full CRE Kd4 = {round(result.params['Kd4_0'].value, 3)} nM,",
        f"half CRE Kd4 = {round(result.params['Kd4_1'].value, 3)} nM,",
        f"no CRE Kd4 = {round(result.params['Kd4_2'].value)} nM",
    ])
    format_bokeh_plot(p)
    p.title.text_font_size = "10.5pt"

    plots.append(p)

bokeh.io.show(bokeh.layouts.column(plots))

---

### Fluorescence anisotropy model

A very similar model can be applied to the fluorescence anisotropy data collected in Chapter 4. In this experiment, the DNA concentration was held constant instead of the protein concentration, and the fraction of DNA bound was reported. The conservation of mass equations are identical to the previous model. Again, we assume that $M_1 DNA$ has a concentration of 0, making the value we want to calculate:

<font size="4">$f = \frac{[M]^2}{K_{d3}K_{d4} + [M]^2}$</font>

In [20]:
anisotropy_model = make_model_function(
    equations=lambda M_total, DNA_total, Kd3, Kd4, M, DNA: [
        M + 2 * (M ** 2) / Kd3 + 2 * (M ** 2) * DNA / (Kd3 * Kd4) - M_total,
        DNA + (M ** 2 * DNA / (Kd3 * Kd4)) - DNA_total,
        (M ** 2) / (Kd3 * Kd4 + (M ** 2)),
    ],
    axis="M_total",
)

anisotropy_model.__doc__ += (
    "\nCalculate fraction DNA bound with stepwise model."
    "\n\nParameters"
    "\n----------"
    "\nM_total : total concentration of monomer (independent variable)"
    "\nDNA_total : total concentration of DNA"
    "\nKd3 : association constant for M + M -> M(2)"
    "\nKd4 : association constant for M(2) + DNA -> M(2)DNA"
    "\nM : concentration of free monomer. Will be calculated"
    "\nDNA : concentration of free DNA. Will be calculated"
)

### Import data from Chapter 4, Figure 4.3: Fluorescence anisotropy experiments of bZIP binding to nsDNA

In [21]:
ns_anisotropy = pd.DataFrame(
    {
        ("14bp nsDNA", "mean"): [
            0.258005,
            0.260189,
            0.258941,
            0.258151,
            0.259778,
            0.263899,
            0.264063,
            0.267293,
            0.271072,
            0.273983,
            0.278454,
            0.287154,
            0.291044,
            0.295333,
            0.30002,
            0.303308,
            0.309141,
            0.309066,
            0.309227,
            0.312791,
            0.309271,
            0.311718,
            0.311824,
        ],
        ("14bp nsDNA", "std"): [
            0.003724,
            0.003427,
            0.003611,
            0.003582,
            0.004098,
            0.003892,
            0.0034,
            0.003441,
            0.005073,
            0.003713,
            0.005141,
            0.006345,
            0.004265,
            0.005823,
            0.003786,
            0.003714,
            0.00393,
            0.004003,
            0.003986,
            0.004025,
            0.003294,
            0.004097,
            0.005822,
        ],
        ("14bp nsDNA", "N"): [170] * 23,
        ("18bp nsDNA", "mean"): [
            0.260491,
            0.263452,
            0.263041,
            0.260602,
            0.261249,
            0.259517,
            0.264488,
            0.267496,
            0.26871,
            0.27288,
            0.274821,
            0.276326,
            0.285035,
            0.287818,
            0.291238,
            0.293414,
            0.294696,
            0.299282,
            0.297454,
            0.297731,
            0.301362,
            0.299422,
            0.304099,
        ],
        ("18bp nsDNA", "std"): [
            0.007197,
            0.006854,
            0.00868,
            0.005782,
            0.005394,
            0.005307,
            0.005322,
            0.006112,
            0.005501,
            0.005605,
            0.00556,
            0.005625,
            0.004883,
            0.007264,
            0.003858,
            0.007363,
            0.006976,
            0.005061,
            0.007732,
            0.005261,
            0.005817,
            0.006127,
            0.005323,
        ],
        ("18bp nsDNA", "N"): [100] * 23,
    },
    index=[
        0,
        0.95,
        3.01,
        9.5,
        12.7,
        16.9,
        22.6,
        30.1,
        40.1,
        53.5,
        71.3,
        95,
        126.7,
        168.9,
        225.3,
        300.3,
        400.5,
        533.9,
        711.9,
        949,
        1266,
        1688,
        2250,
    ],
)

ns_anisotropy.index.name = "[H6GB1 bZIP] (nM)"

# plot imported data


def plot_anisotropy_data(height=400, width=600):

    p = bokeh.plotting.Figure(
        x_axis_type="log",
        x_axis_label="[bZIP] (nM)",
        y_axis_label="Anisotropy",
        height=height,
        width=width,
    )

    for i, DNA in enumerate(ns_anisotropy.columns.levels[0]):
        p.circle(
            ns_anisotropy[DNA].index,
            ns_anisotropy[DNA, "mean"],
            color=palette[i],
            legend_label=DNA,
            size=8,
        )

        # plot error bars
        for _, row in ns_anisotropy[DNA].iterrows():
            p.line(
                [row.name] * 2,
                [row["mean"] - row["std"], row["mean"] + row["std"]],
                color=palette[i],
                legend_label=DNA,
            )

    return p


p = plot_anisotropy_data()
format_bokeh_plot(p)
p.legend.location = "bottom_right"
bokeh.io.show(p)

### Define the parameters and fit the model

This experiment includes two datasets, so we will set up two of each parameter. Paramaters for 14bp nsDNA will have the suffix \"_0\", while parameters for 18bp nsDNA will have the suffix \"_1\". We set DNA_total to be a constant, and we again set $K_{d3}$ to be equal to 33 nM.

In [22]:
anisotropy_params = lmfit.Parameters()

for i, _ in enumerate(ns_anisotropy.columns.levels[0]):
    anisotropy_params.add(f"Kd3_{i}", value=33, min=12, max=55, vary=False)
    anisotropy_params.add(f"Kd4_{i}", value=50, min=0)
    anisotropy_params.add(f"DNA_total_{i}", value=30, vary=False)
    anisotropy_params.add(f"ymin_{i}", value=0.26)
    anisotropy_params.add(f"ymax_{i}", value=0.31)

    if i > 0:
        anisotropy_params[f"Kd3_{i}"].expr = "Kd3_0"

result = lmfit.minimize(
    multidimensional_residuals,
    anisotropy_params,
    args=(anisotropy_model, ns_anisotropy),
)

result

C:\Users\Emily\anaconda3\lib\site-packages\scipy\optimize\minpack.py:175: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)


In [23]:
smooth_x = np.logspace(0, 4, 1000)

p = plot_anisotropy_data()

for i, DNA in enumerate(ns_anisotropy.columns.levels[0]):
    kwargs = {
        "Kd3": result.params[f"Kd3_{i}"],
        "Kd4": result.params[f"Kd4_{i}"],
        "DNA_total": result.params[f"DNA_total_{i}"],
        "ymin": result.params[f"ymin_{i}"],
        "ymax": result.params[f"ymax_{i}"],
    }

    curve = anisotropy_model(smooth_x, **kwargs)
    p.line(
        smooth_x, curve, color=palette[i], legend_label=str(DNA), line_width=2
    )

p.legend.location = "bottom_right"
p.title.text = " ".join([
    f"14bp nsDNA Kd4 = {round(result.params['Kd4_0'].value, 1)} nM;"
    f"18bp nsDNA Kd4 = {round(result.params['Kd4_1'].value, 1)} nM"
])
format_bokeh_plot(p)
p.title.text_font_size = "14pt"

bokeh.io.show(p)

----

# Simulations of stepwise binding constants

As we have already [briefly demonstrated](#Example-model), the models described here can be used for simulating results from certain inputs as well as fitting experimental data. We can use simulations like these to explore the experimental space.

Our experiments have varied either the total monomer concentration or the total DNA concentration. Here, we will design models that can handle these variables changing independently and calculate both the fraction DNA bound and the fraction monomer bound. The total concentrations will vary along the x and y axes, and the fraction bound will be represented by a color-coded heatmap. When building the model, as the "last" equation, we will now provide a list of equations to calculate: first, the fraction of monomer bound, then the fraction of DNA bound.

<font size="5">$f_{M bound} = \frac{M_1DNA + 2M_2DNA}{M + M_1DNA + 2M_2 + 2M_2DNA} = \frac{\frac{[DNA]}{2sK_{d4}} + 2\frac{[M][DNA]}{K_{d3}K_{d4}}}{1 + \frac{[DNA]}{2sK_{d4}} + 2\frac{[M]}{K_{d3}} + 2\frac{[M][DNA]}{K_{d3}K_{d4}}}$</font>


<font size="5">$f_{DNA bound} = \frac{M_1DNA + M_2DNA}{D + M_1DNA + 2M_2DNA} = \frac{\frac{[M]}{2sK_{d4}} + \frac{[M]^2}{K_{d3}K_{d4}}}{1 + \frac{[M]}{2sK_{d4}} + \frac{[M]^2}{K_{d3}K_{d4}}}$</font>

We have taken advantage of the fact that $K_{d1} = 2sK_{d4}$. If three affinity constants are known, the fourth can be calculated.

Let's compare simulations for several values of s.

In [24]:
total_binding_model = make_model_function(
    equations=lambda M_total, DNA_total, Kd3, Kd4, s, M, DNA: [
        M
        + (2 * M ** 2 / Kd3)
        + (M * DNA / (2 * s * Kd4))
        + (2 * DNA * M ** 2 / (Kd3 * Kd4))
        - M_total,
        DNA + (M * DNA / Kd1) + (DNA * M ** 2 / (Kd3 * Kd4)) - DNA_total,
        [
            (DNA / (2 * s * Kd4) + 2 * M * DNA / (Kd3 * Kd4))
            / (
                1 + DNA / (2 * s * Kd4) + 2 * M / Kd3 + 2 * M * DNA / (Kd3 * Kd4)
            ),
            (M / (2 * s * Kd4) + M ** 2 / (Kd3 * Kd4))
            / (1 + M / (2 * s * Kd4) + M ** 2 / (Kd3 * Kd4)),
        ],
    ],
    axis=["M_total", "DNA_total"],
)

The next code block sets up the Kds that will be modeled. Changing these values will change the plots generated in the following block.

In [25]:
# modify these numbers to change the simulation parameters
Kd3 = 33
Kd4 = 34
s_values = [1, 5, 10, 100, 1000]

for s in s_values:
    Kd1 = 2*s*Kd4
    Kd2 = Kd3 / (2*s)
    
    print(f"s={s}")
    print(f"Kd1 = {Kd1}")
    print(f"Kd2 = {Kd2}")
    print()

# axis ranges, given in terms of powers of 10
# (axis start, axis end, number of points)
M_points = np.logspace(1, 4, 50)
DNA_points = np.logspace(1, 4, 50)

s=1
Kd1 = 68
Kd2 = 16.5

s=5
Kd1 = 340
Kd2 = 3.3

s=10
Kd1 = 680
Kd2 = 1.65

s=100
Kd1 = 6800
Kd2 = 0.165

s=1000
Kd1 = 68000
Kd2 = 0.0165



In [26]:
# number of colors in the map
divisions = 20

# generate x and y axis values (monomer and DNA concentrations)
grid = []
for M in M_points:
    grid += [(M, DNA) for DNA in DNA_points]

axis_df = pd.DataFrame(grid, columns=["M_total", "DNA_total"])

results = {}
plots = []

for s in s_values:
    result = total_binding_model(grid, Kd3=Kd3, Kd4=Kd4, s=s)
    results[f"s={s}"] = result
    df = pd.concat(
        [
            axis_df,
            pd.DataFrame(
                result, columns=["fraction monomer bound", "fraction DNA bound"]
            ),
        ],
        axis=1,
    )
    df.loc[df["fraction monomer bound"] > 1, "fraction monomer bound"] = 0.999999

    df["monomer group"] = round(
        (df["fraction monomer bound"] - (1 / (2 * divisions))) * (divisions)
    )
    df["DNA group"] = round(
        (df["fraction DNA bound"] - (1 / (2 * divisions))) * (divisions)
    )

    for i, row in df.iterrows():
        df.loc[i, "monomer group"] = bokeh.palettes.viridis(divisions)[
            int(row["monomer group"])
        ]
        df.loc[i, "DNA group"] = bokeh.palettes.viridis(divisions)[
            int(row["DNA group"])
        ]

    p = bokeh.plotting.Figure(
        title=f"s={s}, fraction monomer bound",
        x_axis_type="log",
        y_axis_type="log",
        x_axis_label="total monomer",
        y_axis_label="total DNA",
    )
    q = bokeh.plotting.Figure(
        title=f"s={s}, fraction DNA bound",
        x_axis_type="log",
        y_axis_type="log",
        x_axis_label="total monomer",
        y_axis_label="total DNA",
    )
    r = bokeh.plotting.Figure(height=600, width=100, y_range=(0, 1))
    for i, color in enumerate(bokeh.palettes.viridis(divisions)):
        p.square(
            df.loc[df["monomer group"] == color, "M_total"],
            df.loc[df["monomer group"] == color, "DNA_total"],
            color=color,
            size=9,
        )
        q.square(
            df.loc[df["DNA group"] == color, "M_total"],
            df.loc[df["DNA group"] == color, "DNA_total"],
            color=color,
            size=9,
        )
        r.square(1, i / divisions + 1 / (2 * divisions), color=color, size=25)

    for plot in [p, q, r]:
        format_bokeh_plot(plot, grid=True)

    r.xaxis.major_label_text_font_size = "0pt"
    r.xaxis.major_tick_line_color = None
    r.yaxis.ticker = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

    plots.append(r)
    plots.append(p)
    plots.append(q)

print("\n".join([
    "",
    "These models often show oddities at a few points, which we discount as errors in numerical root solving.",
    "The goal of the simulation is to consider the overall shape of the heatmaps.",
    ""
]))
bokeh.io.show(bokeh.layouts.grid(plots, ncols=3))


These models often show oddities at a few points, which we discount as errors in numerical root solving.
The goal of the simulation is to consider the overall shape of the heatmaps.



In these simulations, the fraction of DNA bound appears very similar across all values of $s$. In contrast, the fraction of monomer bound clearly changes shape with increasing s, up to a limit < 100 beyond which further increases do not significantly change the heatmap. The difference is most apparent at low concentrations of monomer. 

From this, we can interpret that in future experiments to measure $s$ (or, equivalently, $[M_1DNA]$ or $Kd1$), we should measure the fraction of monomer bound, but that very high values of s may be difficult to distinguish.

Instead of simulating the total monomer bound to DNA, we can also simulate the total monomer dimerized, as measured in Chapter 5, Figure 5.3J:

In [27]:
# protein dimerization version
total_dimerization_model = make_model_function(
    equations=lambda M_total, DNA_total, Kd3, Kd4, s, M, DNA: [
        M
        + (2 * M ** 2 / Kd3)
        + (M * DNA / (2 * s * Kd4))
        + (2 * DNA * M ** 2 / (Kd3 * Kd4))
        - M_total,
        DNA + (M * DNA / Kd1) + (DNA * M ** 2 / (Kd3 * Kd4)) - DNA_total,
        [
            (2 * M / Kd3 + 2 * M * DNA / (Kd3 * Kd4))
            / (
                1 + DNA / (2 * s * Kd4) + 2 * M / Kd3 + 2 * M * DNA / (Kd3 * Kd4)
            ),
            (M / (2 * s * Kd4) + M ** 2 / (Kd3 * Kd4))
            / (1 + M / (2 * s * Kd4) + M ** 2 / (Kd3 * Kd4)),
        ],
    ],
    axis=["M_total", "DNA_total"],
)

In [28]:
# modify these numbers to change the simulation parameters
Kd3 = 33
Kd4 = 34
s_values = [1, 5, 10, 100, 1000]

for s in s_values:
    Kd1 = 2*s*Kd4
    Kd2 = Kd3 / (2*s)
    
    print(f"s={s}")
    print(f"Kd1 = {Kd1}")
    print(f"Kd2 = {Kd2}")
    print()

# axis ranges, given in terms of powers of 10
# (axis start, axis end, number of points)
M_points = np.logspace(-0.5, 2.5, 50)
DNA_points = np.logspace(1, 4, 50)

s=1
Kd1 = 68
Kd2 = 16.5

s=5
Kd1 = 340
Kd2 = 3.3

s=10
Kd1 = 680
Kd2 = 1.65

s=100
Kd1 = 6800
Kd2 = 0.165

s=1000
Kd1 = 68000
Kd2 = 0.0165



In [29]:
# number of colors in the map
divisions = 20

# generate x and y axis values (monomer and DNA concentrations)
grid = []
for M in M_points:
    grid += [(M, DNA) for DNA in DNA_points]

axis_df = pd.DataFrame(grid, columns=["M_total", "DNA_total"])

results = {}
plots = []

for s in s_values:
    result = total_dimerization_model(grid, Kd3=Kd3, Kd4=Kd4, s=s)
    results[f"s={s}"] = result
    df = pd.concat(
        [
            axis_df,
            pd.DataFrame(
                result, columns=["fraction dimer", "fraction DNA bound"]
            ),
        ],
        axis=1,
    )
    df.loc[df["fraction dimer"] > 1, "fraction dimer"] = 0.999999

    df["monomer group"] = round(
        (df["fraction dimer"] - (1 / (2 * divisions))) * (divisions)
    )

    for i, row in df.iterrows():
        df.loc[i, "monomer group"] = bokeh.palettes.viridis(divisions)[
            int(row["monomer group"])
        ]

    p = bokeh.plotting.Figure(
        title=f"s={s}, fraction dimer",
        x_axis_type="log",
        y_axis_type="log",
        x_axis_label="total monomer",
        y_axis_label="total DNA",
    )

    r = bokeh.plotting.Figure(height=600, width=100, y_range=(0, 1))
    for i, color in enumerate(bokeh.palettes.viridis(divisions)):
        p.square(
            df.loc[df["monomer group"] == color, "M_total"],
            df.loc[df["monomer group"] == color, "DNA_total"],
            color=color,
            size=9,
        )

        r.square(1, i / divisions + 1 / (2 * divisions), color=color, size=25)

    for plot in [p, r]:
        format_bokeh_plot(plot, grid=True)

    r.xaxis.major_label_text_font_size = "0pt"
    r.xaxis.major_tick_line_color = None
    r.yaxis.ticker = [0, 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1]

    plots.append(r)
    plots.append(p)
#     plots.append(q)

print("\n".join([
    "",
    "These models often show oddities at a few points, which we discount as errors in numerical root solving.",
    "The goal of the simulation is to consider the overall shape of the heatmaps.",
    ""
]))
bokeh.io.show(bokeh.layouts.grid(plots, ncols=2))


These models often show oddities at a few points, which we discount as errors in numerical root solving.
The goal of the simulation is to consider the overall shape of the heatmaps.

